In [93]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fraud-detection/fraudTest.csv
/kaggle/input/fraud-detection/fraudTrain.csv


In [94]:
# Importing Libraries
import numpy as np
import pandas as pd
from sklearn import metrics
from imblearn.over_sampling import SMOTE

In [95]:
# Importing Datasets
test_df=pd.read_csv("/kaggle/input/fraud-detection/fraudTest.csv")
train_df=pd.read_csv("/kaggle/input/fraud-detection/fraudTrain.csv")

# Data Analysing

In [96]:
train_df.head().T

,0,1,2,3,4
Unnamed: 0,0,1,2,3,4
trans_date_trans_time,2019-01-01 00:00:18,2019-01-01 00:00:44,2019-01-01 00:00:51,2019-01-01 00:01:16,2019-01-01 00:03:06
cc_num,2703186189652095,630423337322,38859492057661,3534093764340240,375534208663984
merchant,"fraud_Rippin, Kub and Mann","fraud_Heller, Gutmann and Zieme",fraud_Lind-Buckridge,"fraud_Kutch, Hermiston and Farrell",fraud_Keeling-Crist
category,misc_net,grocery_pos,entertainment,gas_transport,misc_pos
amt,4.97,107.23,220.11,45.0,41.96
first,Jennifer,Stephanie,Edward,Jeremy,Tyler
last,Banks,Gill,Sanchez,White,Garcia
gender,F,F,M,M,M
street,561 Perry Cove,43039 Riley Greens Suite 393,594 White Dale Suite 530,9443 Cynthia Court Apt. 038,408 Bradley Rest


In [97]:
train_df.isnull().sum()

Unnamed: 0               0
trans_date_trans_time    0
cc_num                   0
merchant                 0
category                 0
amt                      0
first                    0
last                     0
gender                   0
street                   0
city                     0
state                    0
zip                      0
lat                      0
long                     0
city_pop                 0
job                      0
dob                      0
trans_num                0
unix_time                0
merch_lat                0
merch_long               0
is_fraud                 0
dtype: int64

In [98]:
train_df.describe()

,Unnamed: 0,cc_num,amt,zip,lat,long,city_pop,unix_time,merch_lat,merch_long,is_fraud
count,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06,1.296675e+06
mean,6.483370e+05,4.171920e+17,7.035104e+01,4.880067e+04,3.853762e+01,-9.022634e+01,8.882444e+04,1.349244e+09,3.853734e+01,-9.022646e+01,5.788652e-03
std,3.743180e+05,1.308806e+18,1.603160e+02,2.689322e+04,5.075808e+00,1.375908e+01,3.019564e+05,1.284128e+07,5.109788e+00,1.377109e+01,7.586269e-02
min,0.000000e+00,6.041621e+10,1.000000e+00,1.257000e+03,2.002710e+01,-1.656723e+02,2.300000e+01,1.325376e+09,1.902779e+01,-1.666712e+02,0.000000e+00
25%,3.241685e+05,1.800429e+14,9.650000e+00,2.623700e+04,3.462050e+01,-9.679800e+01,7.430000e+02,1.338751e+09,3.473357e+01,-9.689728e+01,0.000000e+00
50%,6.483370e+05,3.521417e+15,4.752000e+01,4.817400e+04,3.935430e+01,-8.747690e+01,2.456000e+03,1.349250e+09,3.936568e+01,-8.743839e+01,0.000000e+00
75%,9.725055e+05,4.642255e+15,8.314000e+01,7.204200e+04,4.194040e+01,-8.015800e+01,2.032800e+04,1.359385e+09,4.195716e+01,-8.023680e+01,0.000000e+00
max,1.296674e+06,4.992346e+18,2.894890e+04,9.978300e+04,6.669330e+01,-6.795030e+01,2.906700e+06,1.371817e+09,6.751027e+01,-6.695090e+01,1.000000e+00


In [99]:
train_df.info(),test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

(None, None)

In [101]:
train_df.is_fraud.value_counts()

is_fraud
0    1289169
1       7506
Name: count, dtype: int64

# Feature Engineering

In [102]:
drop_columns=['Unnamed: 0','cc_num','merchant','trans_num','unix_time','first','last','street','zip']
train_df.drop(columns=drop_columns,inplace=True)
test_df.drop(columns=drop_columns,inplace=True)

In [103]:
train_df.head().T

,0,1,2,3,4
trans_date_trans_time,2019-01-01 00:00:18,2019-01-01 00:00:44,2019-01-01 00:00:51,2019-01-01 00:01:16,2019-01-01 00:03:06
category,misc_net,grocery_pos,entertainment,gas_transport,misc_pos
amt,4.97,107.23,220.11,45.0,41.96
gender,F,F,M,M,M
city,Moravian Falls,Orient,Malad City,Boulder,Doe Hill
state,NC,WA,ID,MT,VA
lat,36.0788,48.8878,42.1808,46.2306,38.4207
long,-81.1781,-118.2105,-112.262,-112.1138,-79.4629
city_pop,3495,149,4154,1939,99
job,"Psychologist, counselling",Special educational needs teacher,Nature conservation officer,Patent attorney,Dance movement psychotherapist


In [104]:
train_df['trans_date_trans_time']=pd.to_datetime(train_df['trans_date_trans_time']) ##Conversion of datatype to datetime64[ns]
train_df['trans_date']=train_df['trans_date_trans_time'].dt.strftime('%Y-%m-%d') ##Extracr date
train_df['trans_date']=pd.to_datetime(train_df['trans_date']) ##Conversion of datatype to datetime64[ns] 
train_df['dob']=pd.to_datetime(train_df['dob']) ##Conversion of datatype to datetime64[ns] 

test_df['trans_date_trans_time']=pd.to_datetime(test_df['trans_date_trans_time'])
test_df['trans_date']=test_df['trans_date_trans_time'].dt.strftime('%Y-%m-%d')
test_df['trans_date']=pd.to_datetime(test_df['trans_date'])
test_df['dob']=pd.to_datetime(test_df['dob'])

In [105]:
train_df['tran_year']=pd.DatetimeIndex(train_df['trans_date']).year
train_df['tran_month']=pd.DatetimeIndex(train_df['trans_date']).month

test_df['tran_year']=pd.DatetimeIndex(test_df['trans_date']).year
test_df['tran_month']=pd.DatetimeIndex(test_df['trans_date']).month

In [106]:
train_df.head()

,trans_date_trans_time,category,amt,gender,city,state,lat,long,city_pop,job,dob,merch_lat,merch_long,is_fraud,trans_date,tran_year,tran_month
0,2019-01-01 00:00:18,misc_net,4.97,F,Moravian Falls,NC,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,36.011293,-82.048315,0,2019-01-01,2019,1
1,2019-01-01 00:00:44,grocery_pos,107.23,F,Orient,WA,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,49.159047,-118.186462,0,2019-01-01,2019,1
2,2019-01-01 00:00:51,entertainment,220.11,M,Malad City,ID,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,43.150704,-112.154481,0,2019-01-01,2019,1
3,2019-01-01 00:01:16,gas_transport,45.00,M,Boulder,MT,46.2306,-112.1138,1939,Patent attorney,1967-01-12,47.034331,-112.561071,0,2019-01-01,2019,1
4,2019-01-01 00:03:06,misc_pos,41.96,M,Doe Hill,VA,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,38.674999,-78.632459,0,2019-01-01,2019,1


In [107]:
#Calculate distance between merchant and home location
train_df['latitudinal_distance'] = abs(round(train_df['merch_lat']-train_df['lat'],3))
train_df['longitudinal_distance'] = abs(round(train_df['merch_long']-train_df['long'],3))

test_df['latitudinal_distance'] = abs(round(test_df['merch_lat']-test_df['lat'],3))
test_df['longitudinal_distance'] = abs(round(test_df['merch_long']-test_df['long'],3))

In [108]:
#Drop Columns that are not relevant to predicy fraud transaction
drop_columns = ['trans_date_trans_time','city','lat','long','job','dob','merch_lat','merch_long','trans_date','state']
train_df.drop(columns=drop_columns,inplace=True)
test_df.drop(columns=drop_columns,inplace=True)

In [109]:
train_df.gender=train_df.gender.apply(lambda x : 1 if x=='M' else 0)
test_df.gender=test_df.gender.apply(lambda x : 1 if x=='M' else 0)

In [110]:
train_df.head()

,category,amt,gender,city_pop,is_fraud,tran_year,tran_month,latitudinal_distance,longitudinal_distance
0,misc_net,4.97,0,3495,0,2019,1,0.068,0.870
1,grocery_pos,107.23,0,149,0,2019,1,0.271,0.024
2,entertainment,220.11,1,4154,0,2019,1,0.970,0.108
3,gas_transport,45.00,1,1939,0,2019,1,0.804,0.447
4,misc_pos,41.96,1,99,0,2019,1,0.254,0.830


In [111]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 9 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   category               1296675 non-null  object 
 1   amt                    1296675 non-null  float64
 2   gender                 1296675 non-null  int64  
 3   city_pop               1296675 non-null  int64  
 4   is_fraud               1296675 non-null  int64  
 5   tran_year              1296675 non-null  int32  
 6   tran_month             1296675 non-null  int32  
 7   latitudinal_distance   1296675 non-null  float64
 8   longitudinal_distance  1296675 non-null  float64
dtypes: float64(3), int32(2), int64(3), object(1)
memory usage: 79.1+ MB


In [112]:
#One Hot Encoding of Category column
train_df = pd.get_dummies(train_df, columns=['category'], prefix='category')
test_df = pd.get_dummies(test_df, columns=['category'], prefix='category')

test_df = test_df.reindex(columns=train_df.columns, fill_value=0)

In [113]:
train_df.info(),test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 22 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   amt                      1296675 non-null  float64
 1   gender                   1296675 non-null  int64  
 2   city_pop                 1296675 non-null  int64  
 3   is_fraud                 1296675 non-null  int64  
 4   tran_year                1296675 non-null  int32  
 5   tran_month               1296675 non-null  int32  
 6   latitudinal_distance     1296675 non-null  float64
 7   longitudinal_distance    1296675 non-null  float64
 8   category_entertainment   1296675 non-null  bool   
 9   category_food_dining     1296675 non-null  bool   
 10  category_gas_transport   1296675 non-null  bool   
 11  category_grocery_net     1296675 non-null  bool   
 12  category_grocery_pos     1296675 non-null  bool   
 13  category_health_fitness  1296675 non-null 

(None, None)

In [114]:
train_df['is_fraud'].value_counts()

is_fraud
0    1289169
1       7506
Name: count, dtype: int64

# Splitting Data

In [115]:
X_train=train_df.drop('is_fraud',axis=1)
y_train=train_df.is_fraud
X_test=test_df.drop('is_fraud',axis=1)
y_test=test_df.is_fraud

In [116]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((1296675, 21), (1296675,), (555719, 21), (555719,))

# Handling data imbalance

In [117]:
smote=SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)

In [118]:
X_train.shape,y_train.shape

((2578338, 21), (2578338,))

In [119]:
X_train.T

,0,1,2,3,4,5,6,7,8,9,...,2578328,2578329,2578330,2578331,2578332,2578333,2578334,2578335,2578336,2578337
amt,4.97,107.23,220.11,45.0,41.96,94.63,44.54,71.65,4.27,198.39,...,689.93949,856.139695,827.64699,915.245075,1045.678948,242.469916,6.854902,279.170625,922.598941,785.42816
gender,0,0,1,1,1,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,1
city_pop,3495,149,4154,1939,99,2158,2691,6018,1472,151785,...,2351,2090,4292,27829,1920,71485,7420,9815,1577385,47238
tran_year,2019,2019,2019,2019,2019,2019,2019,2019,2019,2019,...,2019,2019,2019,2019,2019,2020,2020,2020,2019,2019
tran_month,1,1,1,1,1,1,1,1,1,1,...,4,7,6,3,5,2,3,2,10,6
latitudinal_distance,0.068,0.271,0.97,0.804,0.254,0.278,0.83,0.105,0.016,0.657,...,0.176142,0.496832,0.609996,0.760493,0.342297,0.928211,0.681006,0.447286,0.366518,0.878651
longitudinal_distance,0.87,0.024,0.108,0.447,0.83,0.948,0.836,0.06,0.297,0.136,...,0.432709,0.104225,0.871403,0.077227,0.058289,0.182248,0.554149,0.126327,0.829732,0.3259
category_entertainment,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
category_food_dining,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
category_gas_transport,False,False,False,True,False,True,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False


# Standarizing Data

In [120]:
from sklearn.preprocessing import StandardScaler

# Step 1: Fit the StandardScaler on the training data
scaler = StandardScaler()
scaler.fit(X_train)

# Step 2: Transform both the training and testing data
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

# Modeling

## DecisionTree

In [121]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

# Initialize and fit the Decision Tree classifier on the training data
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = clf.predict(X_test)

# Generate a classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.49      0.62      0.54      2145

    accuracy                           1.00    555719
   macro avg       0.74      0.81      0.77    555719
weighted avg       1.00      1.00      1.00    555719



## RandomForestClassifier

In [122]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and fit the Random Forest classifier on the training data
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = clf.predict(X_test)

# Generate a classification report
report = classification_report(y_test, y_pred)

# Print the classification report
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    553574
           1       0.69      0.67      0.68      2145

    accuracy                           1.00    555719
   macro avg       0.84      0.84      0.84    555719
weighted avg       1.00      1.00      1.00    555719

